## 1. NHN News title search
* requests, beautifulsoup basic
* css selector -> attribute

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [3]:
url = 'https://news.naver.com/'
response = requests.get(url)
print(f'status code : {response.status_code}')
print(f'response header : {response.headers}')

status code : 200
response header : {'date': 'Mon, 20 Jul 2020 07:55:50 GMT', 'cache-control': 'no-cache', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'set-cookie': 'JSESSIONID=3DE2C104AF3C360165F02D5E8F9D5A49; Path=/main; HttpOnly', 'content-language': 'ko-KR', 'vary': 'Accept-Encoding', 'content-encoding': 'gzip', 'transfer-encoding': 'chunked', 'content-type': 'text/html;charset=EUC-KR', 'referrer-policy': 'unsafe-url', 'server': 'nfront'}


<pre>
http protocol
http method (REST)
    GET: append to url and send that (less secure)
    POST : encapsule with body stream (more secure)
    PUT : 
    DELETE : 

Cookie & Session
    HTTP is connection less
    For each client~ 
    ** HAVE TO DO

CSS Selector
    Tag selector            : select the tag with 'tag name'
                                soup.select('li')
    ID(#) selector          : select the tag with 'id'
                                soup.select('#myDiv') 
    Class(.) selector       : select the tag with 'class'
                                soup.select('.myClass')  OR
                                soup.select('div.myClass')
    Attribute([]) selector   : select the tag with 'attribute'
                                soup.select["a[href='']"]
                                tag[attr*="bar"] : partially matched
                                tag[attr^="bar"] : if start with "bar"
                                tag[attr$="bar"] : if end with "bar"
</pre>                                
```html                            
<div id = 'myDiv'>
</div>
<div class = 'myClass'>
</div>
<ul>
    <li>aaa</li>
    <li>bbb</li>
</ul>
```
    

In [52]:
html_doc = response.text #page source code
## BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

tag_list = soup.select("a[href*=read.nhn]")

# for i in tag_list:
#     ttl = i.get('title') ## get from bs4
#     if ttl is not None: 
#         print(ttl)
# print("===========================")
for i in tag_list:
    ttl = i.text.strip() # text from bs4
    # link = urljoin(url, i['href'])
    # print(f'{ttl} \"{link}\"')
    # print(ttl)


## 2. papago API 
### Code example from Naver

In [5]:
import os
import sys
import urllib.request
client_id = "6YdSTBn2_cC7tlhp7vHZ"
client_secret = "_qvvwbv5BE"
encText = urllib.parse.quote("Why she had to go, I don't know, she wouldn't say.")
data = "source=en&target=ko&text=" + encText
url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"message":{"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0","result":{"srcLangType":"en","tarLangType":"ko","translatedText":"왜 그녀가 가야만 했는지 모르겠다, 그녀는 말하지 않을 것이다.","engineType":"N2MT","pivot":null}}}


### Using Requests , JSON

In [6]:
import requests
client_id = "6YdSTBn2_cC7tlhp7vHZ"
client_secret = "_qvvwbv5BE"
encText = "Why she had to go, I don't know, she wouldn't say."
url = "https://openapi.naver.com/v1/papago/n2mt"

req_headers = {
    "X-Naver-Client-Id" : client_id,
    "X-Naver-Client-Secret" : client_secret,
}
params ={
    "source" : 'en',
    "target" : 'ko',
    "text" : encText,
}
response = requests.post(url, headers = req_headers, data = params)
# print(f'response headers : {response.headers}')
# print(f'request headers : {response.request.headers}')
# print(f'status cdoe : {response.status_code}')

if response.status_code == 200:
    print(response.text)
else:
    print(f'Error Code {response.status_code}')

{"message":{"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0","result":{"srcLangType":"en","tarLangType":"ko","translatedText":"왜 그녀가 가야만 했는지 모르겠다, 그녀는 말하지 않을 것이다.","engineType":"N2MT","pivot":null}}}


In [7]:
# print only translated text
myjson = response.json()
print(myjson['message']['result']['translatedText'])

왜 그녀가 가야만 했는지 모르겠다, 그녀는 말하지 않을 것이다.


### (From requests,) Using requests, session object 

In [15]:
import requests
from requests import Request, Session
client_id = "6YdSTBn2_cC7tlhp7vHZ"
client_secret = "_qvvwbv5BE"
encText = "Oh, I believe in yesterday."
url = "https://openapi.naver.com/v1/papago/n2mt"

req_headers = {
    "X-Naver-Client-Id" : client_id,
    "X-Naver-Client-Secret" : client_secret,
}
params ={
    "source" : 'en',
    "target" : 'ko',
    "text" : encText,
}
## Make session object
session = Session()
## Make Request object
request = Request('POST', url, headers=req_headers, data = params)
## prepare() from request
prepared = request.prepare()

## send() from session, call server
response = session.send(prepared)


if response.status_code == 200:
    print(response.json()['message']['result']['translatedText'])
else:
    print(f'Error Code {response.status_code}')

오, 나는 어제를 다시 그리게 돼.


### Translate whole file

In [51]:
import requests,os,sys
from requests import Request, Session
# print(os.getcwd())
# os.chdir('../docs/webscrap_source/data')
def get_lyrics():
    temp = []
    with open('./yesterday.txt', 'r', encoding = 'utf8') as file:
        contents = file.read()
        temp = contents.split('\n') ## this is a list automatically 
    return temp
    
def save_file(my_list):
    with open('./yesterday_korean.txt', 'w', encoding = 'utf8') as file:
        file.writelines(my_list)


def main():
    session = Session()

    client_id = "6YdSTBn2_cC7tlhp7vHZ"
    client_secret = "_qvvwbv5BE"
    encText = "Oh, I believe in yesterday."
    url = "https://openapi.naver.com/v1/papago/n2mt"

    req_headers = {
        "X-Naver-Client-Id" : client_id,
        "X-Naver-Client-Secret" : client_secret,
    }
    
    lyrics = get_lyrics()
    lyrics = [_ for _ in lyrics if len(_) != 0]
    ko_texts = []
    for encText in lyrics:
        params ={
            "source" : 'en',
            "target" : 'ko',
            "text" : encText,
        }
        req = Request('POST', url, data = params, headers= req_headers)
        prepared = req.prepare()
        res = session.send(prepared)
        try:
            ko_text = res.json()['message']['result']['translatedText']
        except Exception as err:
            print(err)
        else:
            ko_texts.append(ko_text +'\n')
            print(f'EN : {encText}\nKR : {ko_text}\n')
    save_file(ko_texts)
main()

EN : Yesterday all my troubles seemed so far away.
KR : 어제 나의 모든 고민들은 너무나 멀리 떨어져 있는 것 같았다.

EN : Now it looks as though they're here to stay.
KR : 이제 그들은 여기 머물기 위해 온 것처럼 보인다.

EN : Oh, I believe in yesterday.
KR : 오, 나는 어제를 다시 그리게 돼.

EN : Suddenly I'm not half the man I used to be.
KR : 갑자기 나는 예전과 같은 사람이 아니다.

EN : There's a shadow hanging over me.
KR : 내 위에 그림자가 드리워져 있다.

EN : Oh, yesterday came suddenly.
KR : 아, 어제 갑자기 왔구나.

EN : Why she had to go, I don't know, she wouldn't say.
KR : 왜 그녀가 가야만 했는지 모르겠다, 그녀는 말하지 않을 것이다.

EN : I said something wrong, now I long for yesterday.
KR : 내가 말을 잘못해서 어제가 그립다.

EN : Yesterday love was such an easy game to play.
KR : 어제는 사랑이 정말 쉬운 게임이었다.

EN : Now I need a place to hide away.
KR : 이제 나는 숨을 곳이 필요해.

EN : Oh, I believe in yesterday.
KR : 오, 나는 어제를 다시 그리게 돼.

EN : Why she had to go, I don't know, she wouldn't say.
KR : 왜 그녀가 가야만 했는지 모르겠다, 그녀는 말하지 않을 것이다.

EN : I said something wrong, now I long for yesterday.
KR : 내가 말을 잘못해서 어제가 그립다.

EN : Yesterd